In [3]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

import json
from datetime import datetime

In [4]:
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [5]:
projects_df = pd.read_csv('/root/data/projects.csv',
                         parse_dates=['project_created_at', 'project_last_pushed_at', 'project_updated_at'])
grep_df = pd.read_csv('/root/data/lexical/grep_findings_0_499.csv')
package_df = pd.read_csv('/root/data/packages_0_499.csv')
vet_df = pd.read_csv('/root/data/lexical/vet_findings_0_499.csv')
gosec_df = pd.read_csv('/root/data/lexical/gosec_findings_0_499.csv')

In [7]:
grep_df\
    .where(grep_df['match_type']=='unsafe.Pointer')\
    .groupby(['package_import_path', 'project_name'])['package_import_path'].count()\
    .groupby('package_import_path').max()\
    .sort_values(ascending=True)[-20:]\
    .plot.barh(title='Number of unsafe.Pointer usages per package', figsize=(5,5))

plt.savefig('/root/data/figures/unsafe-usages-by-package-n20.pgf')